In [105]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")


In [106]:
import torch
from torch.utils.data import DataLoader, Dataset

class GPTDatasetV1(Dataset):
    def __init__(self, txt, tokenizer, max_length, stride):
        self.input_idx = []
        self.target_idx = []
        token_ids = tokenizer.encode(txt)
        for i in range(0, len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunk = token_ids[i+1:i+max_length+1]
            self.input_idx.append(torch.tensor(input_chunk))
            self.target_idx.append(torch.tensor(target_chunk))
    
    def __len__(self):
        return len(self.input_idx)
    def __getitem__(self, index) :
        return self.input_idx[index], self.target_idx[index]



In [107]:
def create_dataloader_v1(txt, batch_size=4, max_length=256,
 stride=128, shuffle=True, drop_last=True,
 num_workers=0):
    
   tokenizer = tiktoken.get_encoding("gpt2")
   dataset   = GPTDatasetV1(txt, tokenizer, max_length, stride)
   dataloader = DataLoader(
     dataset,
     batch_size=batch_size,
     shuffle=shuffle,
     drop_last=drop_last,
     num_workers=num_workers
 )
   return dataloader

In [108]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
    raw_text = f.read()
dataloader = create_dataloader_v1(raw_text, batch_size=8, max_length=4, stride=4, shuffle=False)
data_iter = iter(dataloader)
inputs,targets = next(data_iter)
print(inputs)

tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11]])


#### Embedding word 

In [109]:
vocab_size = 50257 
output_dimensions= 256
max_length=4
token_embedding_layer = torch.nn.Embedding(vocab_size,output_dimensions)
print(token_embedding_layer)


Embedding(50257, 256)


In [110]:
token_embeddings = token_embedding_layer(inputs)
print(token_embeddings.shape)

torch.Size([8, 4, 256])


In [111]:
context_length = max_length
pos_embedding_layer = torch.nn.Embedding(context_length,output_dimensions)

print(pos_embedding_layer.weight[0:5])


tensor([[ 1.6101, -0.5680,  1.3296,  ..., -0.9491,  2.6084, -1.3818],
        [-0.3198,  0.0304,  2.2102,  ...,  0.1460, -1.0753,  0.6180],
        [-0.4907,  0.4497,  0.9035,  ...,  0.9045, -0.4014, -1.0983],
        [-0.6530,  1.6669,  0.3676,  ...,  0.6280, -0.6195, -2.3684]],
       grad_fn=<SliceBackward0>)


In [112]:
posittion = pos_embedding_layer(torch.arange(context_length))
print(posittion)
input_embeddings = token_embeddings + posittion
print(input_embeddings.shape)


tensor([[ 1.6101, -0.5680,  1.3296,  ..., -0.9491,  2.6084, -1.3818],
        [-0.3198,  0.0304,  2.2102,  ...,  0.1460, -1.0753,  0.6180],
        [-0.4907,  0.4497,  0.9035,  ...,  0.9045, -0.4014, -1.0983],
        [-0.6530,  1.6669,  0.3676,  ...,  0.6280, -0.6195, -2.3684]],
       grad_fn=<EmbeddingBackward0>)
torch.Size([8, 4, 256])


In [113]:
embedding = torch.nn.Embedding(10, 3)
# a batch of 2 samples of 4 indices each
print(embedding)
input = torch.LongTensor([[1, 2, 4, 5], [4, 3, 2, 9]])
embedding(input)


Embedding(10, 3)


tensor([[[-1.1514, -0.8176,  2.4795],
         [-1.2571, -0.3281, -0.0561],
         [-0.1321, -1.2943, -0.6398],
         [-0.1070, -0.9359, -0.7723]],

        [[-0.1321, -1.2943, -0.6398],
         [-1.6411,  0.5236, -0.4785],
         [-1.2571, -0.3281, -0.0561],
         [-2.5452,  1.1331,  1.1942]]], grad_fn=<EmbeddingBackward0>)